In [ ]:
import mlflow

MLFLOW_URL = "https://fleecier-rufus-decadently.ngrok-free.dev" # This URL might need to be updated as ngrok URLs are temporary.

mlflow.set_tracking_uri(MLFLOW_URL)
mlflow.set_experiment("Exp2-DDPM-PixelSpace")

print(f"[INFO] MLflow tracking URI set to: {MLFLOW_URL}")
print(f"[INFO] MLflow experiment set to: Exp2-DDPM-PixelSpace")

[INFO] MLflow tracking URI set to: https://fleecier-rufus-decadently.ngrok-free.dev
[INFO] MLflow experiment set to: Exp2-DDPM-PixelSpace


In [ ]:
npz_path = "./data/intermediate/pixel_art_data.npz"

data = np.load(npz_path)
images = data["images"]        # (N, 16,16,3), float32 [0,1]
labels = data["labels"]        # (N,)  (no se usan aquí)
print("Dataset:", images.shape)

# Convertir a tensor
images_tf = tf.convert_to_tensor(images, dtype=tf.float32)


IMG_SHAPE = (16, 16, 3)
T = 300                    # pasos de difusión
BATCH_SIZE = 256
EPOCHS = 20
LR = 2e-4

# Schedule lineal de betas
beta_start = 1e-4
beta_end    = 2e-2
betas = np.linspace(beta_start, beta_end, T, dtype=np.float32)

alphas = 1.0 - betas
alphas_cumprod = np.cumprod(alphas, axis=0)

# Precomputos útiles
sqrt_alphas_cumprod = np.sqrt(alphas_cumprod)
sqrt_one_minus_alphas_cumprod = np.sqrt(1.0 - alphas_cumprod)
sqrt_recip_alphas = np.sqrt(1.0 / alphas)

betas_tf = tf.constant(betas)
alphas_tf = tf.constant(alphas)
alphas_cumprod_tf = tf.constant(alphas_cumprod)
sqrt_alphas_cumprod_tf = tf.constant(sqrt_alphas_cumprod)
sqrt_one_minus_alphas_cumprod_tf = tf.constant(sqrt_one_minus_alphas_cumprod)
sqrt_recip_alphas_tf = tf.constant(sqrt_recip_alphas)


num_samples = images_tf.shape[0]
train_dataset = tf.data.Dataset.from_tensor_slices(images_tf)
train_dataset = train_dataset.shuffle(10000).batch(BATCH_SIZE, drop_remainder=True)

print("[INFO] Dataset NPZ cargado y configuraciones iniciales establecidas.")

Dataset: (89400, 16, 16, 3)
[INFO] Dataset NPZ cargado y configuraciones iniciales establecidas.


In [ ]:
import tensorflow as tf
from tensorflow.keras import layers
import numpy as np

# Custom Keras Layer for timestep embedding to handle KerasTensors
class TimeEmbedding(layers.Layer):
    def __init__(self, dim, **kwargs):
        super().__init__(**kwargs)
        self.dim = dim
        self.half = dim // 2
        # freqs can be precomputed as it only depends on `dim` which is static
        freqs = tf.exp(
            tf.range(self.half, dtype=tf.float32) * -np.log(10000.0) / (self.half - 1)
        )
        self.freqs = tf.constant(freqs, dtype=tf.float32)

    def call(self, timesteps):
        timesteps = tf.cast(timesteps, tf.float32)[:, None]  # (B,1)
        args = timesteps * self.freqs[None, :]  # (B, half)
        emb = tf.concat([tf.sin(args), tf.cos(args)], axis=-1)
        if self.dim % 2 == 1:
            emb = tf.pad(emb, [[0, 0], [0, 1]])
        return emb  # (B, dim)

# Custom Keras Layer for adding time embedding to feature maps
class AddTimeLayer(layers.Layer):
    def __init__(self, temb_proj_channels, **kwargs):
        super().__init__(**kwargs)
        self.temb_proj_channels = temb_proj_channels
        self.dense_projection = layers.Dense(self.temb_proj_channels)
        self.activation = layers.Activation("relu")

    def call(self, inputs):
        x, temb = inputs # x is feature map, temb is time embedding
        h = self.dense_projection(temb)
        h = self.activation(h)
        # Reshape to allow broadcasting with feature maps
        h = tf.reshape(h, (-1, 1, 1, self.temb_proj_channels))
        return x + h

print("[INFO] TimeEmbedding and AddTimeLayer classes defined.")

[INFO] TimeEmbedding and AddTimeLayer classes defined.


In [ ]:
import tensorflow as tf

def extract(a, t, x_shape):
    """
    Extrae elementos de un vector 'a' en los índices t y les da forma
    broadcasting-friendly con x_shape.
    """
    bs = tf.shape(t)[0]
    out = tf.gather(a, t)
    out = tf.reshape(out, (bs, 1, 1, 1))
    return tf.broadcast_to(out, x_shape)

def q_sample(x0, t, noise):
    """
    q(x_t | x_0) = sqrt(a_bar_t) * x0 + sqrt(1 - a_bar_t) * noise
    """
    # Ensure these are defined in the global scope or passed as arguments if not.
    # For this execution, they are assumed to be in the kernel's global scope from previous cells.
    # sqrt_alphas_cumprod_tf and sqrt_one_minus_alphas_cumprod_tf

    sqrt_ab = extract(sqrt_alphas_cumprod_tf, t, tf.shape(x0))
    sqrt_1mab = extract(sqrt_one_minus_alphas_cumprod_tf, t, tf.shape(x0))
    return sqrt_ab * x0 + sqrt_1mab * noise

print("[INFO] Diffusion helper functions `extract` and `q_sample` defined.")

[INFO] Diffusion helper functions `extract` and `q_sample` defined.


In [ ]:
def build_unet(time_dim=64, base_channels=32):
    x_in = layers.Input(shape=IMG_SHAPE, name="x")
    t_in = layers.Input(shape=(), dtype=tf.int32, name="t")

    # Time embedding using the new custom layer
    temb = TimeEmbedding(dim=time_dim)(t_in)
    temb = layers.Dense(time_dim, activation="relu")(temb)
    temb = layers.Dense(time_dim, activation="relu")(temb)

    # Encoder
    h1 = layers.Conv2D(base_channels, 3, padding="same", activation="relu")(x_in)
    h1 = layers.Conv2D(base_channels, 3, padding="same", activation="relu")(h1)
    h1 = AddTimeLayer(base_channels)([h1, temb]) # Use the new layer
    d1 = layers.MaxPooling2D(2)(h1)  # 16 -> 8

    h2 = layers.Conv2D(base_channels*2, 3, padding="same", activation="relu")(d1)
    h2 = layers.Conv2D(base_channels*2, 3, padding="same", activation="relu")(h2)
    h2 = AddTimeLayer(base_channels*2)([h2, temb]) # Use the new layer
    d2 = layers.MaxPooling2D(2)(h2)  # 8 -> 4

    # Bottleneck
    b = layers.Conv2D(base_channels*4, 3, padding="same", activation="relu")(d2)
    b = layers.Conv2D(base_channels*4, 3, padding="same", activation="relu")(b)
    b = AddTimeLayer(base_channels*4)([b, temb]) # Use the new layer

    # Decoder
    u2 = layers.UpSampling2D()(b)  # 4 -> 8
    u2 = layers.Concatenate()([u2, h2])
    u2 = layers.Conv2D(base_channels*2, 3, padding="same", activation="relu")(u2)
    u2 = layers.Conv2D(base_channels*2, 3, padding="same", activation="relu")(u2)

    u1 = layers.UpSampling2D()(u2)  # 8 -> 16
    u1 = layers.Concatenate()([u1, h1])
    u1 = layers.Conv2D(base_channels, 3, padding="same", activation="relu")(u1)
    u1 = layers.Conv2D(base_channels, 3, padding="same", activation="relu")(u1)

    out = layers.Conv2D(3, 3, padding="same", activation=None)(u1)  # predicción de ruido

    model = models.Model(inputs=[x_in, t_in], outputs=out)
    return model


model = build_unet(time_dim=64, base_channels=32)
optimizer = optimizers.Adam(LR)

print("U-Net Model Summary:")
print(model.summary())

U-Net Model Summary:


Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ t (InputLayer)      │ (None)            │          0 │ -                 │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ x (InputLayer)      │ (None, 16, 16, 3) │          0 │ -                 │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ time_embedding_2    │ (None, 64)        │          0 │ t[0][0]           │
│ (TimeEmbedding)     │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_13 (Conv2D)  │ (None, 16, 16,    │        896 │ x[0][0]           │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_8 (Dense)     │ (None, 64)        │      4,160 │ time_embedding_2… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_14 (Conv2D)  │ (None, 16, 16,    │      9,248 │ conv2d_13[0][0]   │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_9 (Dense)     │ (None, 64)        │      4,160 │ dense_8[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ add_time_layer_3    │ (None, 16, 16,    │      2,080 │ conv2d_14[0][0],  │
│ (AddTimeLayer)      │ 32)               │            │ dense_9[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d_2     │ (None, 8, 8, 32)  │          0 │ add_time_layer_3… │
│ (MaxPooling2D)      │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_15 (Conv2D)  │ (None, 8, 8, 64)  │     18,496 │ max_pooling2d_2[… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_16 (Conv2D)  │ (None, 8, 8, 64)  │     36,928 │ conv2d_15[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ add_time_layer_4    │ (None, 8, 8, 64)  │      4,160 │ conv2d_16[0][0],  │
│ (AddTimeLayer)      │                   │            │ dense_9[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d_3     │ (None, 4, 4, 64)  │          0 │ add_time_layer_4… │
│ (MaxPooling2D)      │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_17 (Conv2D)  │ (None, 4, 4, 128) │     73,856 │ max_pooling2d_3[… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_18 (Conv2D)  │ (None, 4, 4, 128) │    147,584 │ conv2d_17[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ add_time_layer_5    │ (None, 4, 4, 128) │      8,320 │ conv2d_18[0][0],  │
│ (AddTimeLayer)      │                   │            │ dense_9[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ up_sampling2d_2     │ (None, 8, 8, 128) │          0 │ add_time_layer_5… │
│ (UpSampling2D)      │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate_2       │ (None, 8, 8, 192) │          0 │ up_sampling2d_2[… │
│ (Concatenate)       │                   │            │ add_time_layer_4… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_19 (Conv2D)  │ (None, 8, 8, 64)  │    110,656 │ concatenate_2[0]… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_20 (Conv2D)  │ (None, 8, 8, 64)  │     36,928 │ conv2d_19[0][0] 

 Total params: 495,267 (1.89 MB)

 Trainable params: 495,267 (1.89 MB)

 Non-trainable params: 0 (0.00 B)

None


In [ ]:
import os
import numpy as np
import mlflow
import mlflow.keras
import matplotlib.pyplot as plt

import tensorflow as tf
from tensorflow.keras import layers, models, optimizers

from sklearn.decomposition import PCA

# ============================================================
# 1. CONFIG MLflow
# ============================================================

MLFLOW_URL = "https://fleecier-rufus-decadently.ngrok-free.dev"   # <--- AJUSTA ESTO

mlflow.set_tracking_uri(MLFLOW_URL)
mlflow.set_experiment("Exp2-DDPM-PixelSpace")


# ============================================================
# 2. CARGAR NPZ (INTERMEDIATE)
# ============================================================

npz_path = "./data/intermediate/pixel_art_data.npz"  # <-- cambia según dónde lo montes

data = np.load(npz_path)
images = data["images"]        # (N, 16,16,3), float32 [0,1]
labels = data["labels"]        # (N,)  (no se usan aquí)
print("Dataset:", images.shape)

# Convertir a tensor
images_tf = tf.convert_to_tensor(images, dtype=tf.float32)


# ============================================================
# 3. CONFIGURACIÓN DE DDPM
# ============================================================

IMG_SHAPE = (16, 16, 3)
T = 300                    # pasos de difusión
BATCH_SIZE = 256
EPOCHS = 50
LR = 2e-4

# Schedule lineal de betas
beta_start = 1e-4
beta_end   = 2e-2
betas = np.linspace(beta_start, beta_end, T, dtype=np.float32)

alphas = 1.0 - betas
alphas_cumprod = np.cumprod(alphas, axis=0)

# Precomputos útiles
sqrt_alphas_cumprod = np.sqrt(alphas_cumprod)
sqrt_one_minus_alphas_cumprod = np.sqrt(1.0 - alphas_cumprod)
sqrt_recip_alphas = np.sqrt(1.0 / alphas)

betas_tf = tf.constant(betas)
alphas_tf = tf.constant(alphas)
alphas_cumprod_tf = tf.constant(alphas_cumprod)
sqrt_alphas_cumprod_tf = tf.constant(sqrt_alphas_cumprod)
sqrt_one_minus_alphas_cumprod_tf = tf.constant(sqrt_one_minus_alphas_cumprod)
sqrt_recip_alphas_tf = tf.constant(sqrt_recip_alphas)


# ============================================================
# 4. FUNCIONES DE DIFUSIÓN
# ============================================================

def extract(a, t, x_shape):
    """
    Extrae elementos de un vector 'a' en los índices t y les da forma
    broadcasting-friendly con x_shape.
    """
    bs = tf.shape(t)[0]
    out = tf.gather(a, t)
    out = tf.reshape(out, (bs, 1, 1, 1))
    return tf.broadcast_to(out, x_shape)


def q_sample(x0, t, noise):
    """
    q(x_t | x_0) = sqrt(a_bar_t) * x0 + sqrt(1 - a_bar_t) * noise
    """
    sqrt_ab = extract(sqrt_alphas_cumprod_tf, t, tf.shape(x0))
    sqrt_1mab = extract(sqrt_one_minus_alphas_cumprod_tf, t, tf.shape(x0))
    return sqrt_ab * x0 + sqrt_1mab * noise


# ============================================================
# 5. EMBDDING DE TIEMPO (SINUSOIDAL)
# ============================================================

# Custom Keras Layer for timestep embedding to handle KerasTensors
class TimeEmbedding(layers.Layer):
    def __init__(self, dim, **kwargs):
        super().__init__(**kwargs)
        self.dim = dim
        self.half = dim // 2
        # freqs can be precomputed as it only depends on `dim` which is static
        freqs = tf.exp(
            tf.range(self.half, dtype=tf.float32) * -np.log(10000.0) / (self.half - 1)
        )
        self.freqs = tf.constant(freqs, dtype=tf.float32)

    def call(self, timesteps):
        timesteps = tf.cast(timesteps, tf.float32)[:, None]  # (B,1)
        args = timesteps * self.freqs[None, :]  # (B, half)
        emb = tf.concat([tf.sin(args), tf.cos(args)], axis=-1)
        if self.dim % 2 == 1:
            emb = tf.pad(emb, [[0, 0], [0, 1]])
        return emb  # (B, dim)

# Custom Keras Layer for adding time embedding to feature maps
class AddTimeLayer(layers.Layer):
    def __init__(self, temb_proj_channels, **kwargs):
        super().__init__(**kwargs)
        self.temb_proj_channels = temb_proj_channels
        self.dense_projection = layers.Dense(self.temb_proj_channels)
        self.activation = layers.Activation("relu")

    def call(self, inputs):
        x, temb = inputs # x is feature map, temb is time embedding
        h = self.dense_projection(temb)
        h = self.activation(h)
        # Reshape to allow broadcasting with feature maps
        h = tf.reshape(h, (-1, 1, 1, self.temb_proj_channels))
        return x + h


# ============================================================
# 6. U-NET PEQUEÑO CON TIEMPO
# ============================================================

def build_unet(time_dim=64, base_channels=32):
    x_in = layers.Input(shape=IMG_SHAPE, name="x")
    t_in = layers.Input(shape=(), dtype=tf.int32, name="t")

    # Time embedding using the new custom layer
    temb = TimeEmbedding(dim=time_dim)(t_in)
    temb = layers.Dense(time_dim, activation="relu")(temb)
    temb = layers.Dense(time_dim, activation="relu")(temb)

    # Encoder
    h1 = layers.Conv2D(base_channels, 3, padding="same", activation="relu")(x_in)
    h1 = layers.Conv2D(base_channels, 3, padding="same", activation="relu")(h1)
    h1 = AddTimeLayer(base_channels)([h1, temb]) # Use the new layer
    d1 = layers.MaxPooling2D(2)(h1)  # 16 -> 8

    h2 = layers.Conv2D(base_channels*2, 3, padding="same", activation="relu")(d1)
    h2 = layers.Conv2D(base_channels*2, 3, padding="same", activation="relu")(h2)
    h2 = AddTimeLayer(base_channels*2)([h2, temb]) # Use the new layer
    d2 = layers.MaxPooling2D(2)(h2)  # 8 -> 4

    # Bottleneck
    b = layers.Conv2D(base_channels*4, 3, padding="same", activation="relu")(d2)
    b = layers.Conv2D(base_channels*4, 3, padding="same", activation="relu")(b)
    b = AddTimeLayer(base_channels*4)([b, temb]) # Use the new layer

    # Decoder
    u2 = layers.UpSampling2D()(b)  # 4 -> 8
    u2 = layers.Concatenate()([u2, h2])
    u2 = layers.Conv2D(base_channels*2, 3, padding="same", activation="relu")(u2)
    u2 = layers.Conv2D(base_channels*2, 3, padding="same", activation="relu")(u2)

    u1 = layers.UpSampling2D()(u2)  # 8 -> 16
    u1 = layers.Concatenate()([u1, h1])
    u1 = layers.Conv2D(base_channels, 3, padding="same", activation="relu")(u1)
    u1 = layers.Conv2D(base_channels, 3, padding="same", activation="relu")(u1)

    out = layers.Conv2D(3, 3, padding="same", activation=None)(u1)  # predicción de ruido

    model = models.Model(inputs=[x_in, t_in], outputs=out)
    return model


model = build_unet(time_dim=64, base_channels=32)
optimizer = optimizers.Adam(LR)

print(model.summary())


# ============================================================
# 7. DATASET TF PARA ENTRENAR
# ============================================================

num_samples = images_tf.shape[0]
train_dataset = tf.data.Dataset.from_tensor_slices(images_tf)
train_dataset = train_dataset.shuffle(10000).batch(BATCH_SIZE, drop_remainder=True)


# ============================================================
# 8. LOOP DE ENTRENAMIENTO DDPM
# ============================================================

@tf.function
def train_step(x0):
    batch_size = tf.shape(x0)[0]
    t = tf.random.uniform(
        shape=(batch_size,), minval=0, maxval=T, dtype=tf.int32
    )
    noise = tf.random.normal(shape=tf.shape(x0))
    x_t = q_sample(x0, t, noise)

    with tf.GradientTape() as tape:
        noise_pred = model([x_t, t], training=True)
        loss = tf.reduce_mean(tf.square(noise - noise_pred))

    grads = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(grads, model.trainable_variables))
    return loss


def sample_ddpm(num_samples=64):
    """
    Genera muestras desde ruido puro.
    """
    x = tf.random.normal((num_samples,) + IMG_SHAPE)
    for t in reversed(range(T)):
        t_batch = tf.fill((num_samples,), t)
        eps_theta = model([x, t_batch], training=False)

        beta_t = betas_tf[t]
        alpha_t = alphas_tf[t]
        alpha_bar_t = alphas_cumprod_tf[t]

        sqrt_one_minus_ab = tf.sqrt(1.0 - alpha_bar_t)
        sqrt_recip_alpha = tf.sqrt(1.0 / alpha_t)
        sqrt_beta = tf.sqrt(beta_t)

        # Ecuación de mu_t
        x0_pred = (x - sqrt_one_minus_ab * eps_theta) / tf.sqrt(alpha_bar_t)
        mean = sqrt_recip_alpha * (x - beta_t / sqrt_one_minus_ab * eps_theta)

        if t > 0:
            z = tf.random.normal(tf.shape(x))
            x = mean + sqrt_beta * z
        else:
            x = mean
    x = tf.clip_by_value(x, 0.0, 1.0)
    return x


# ============================================================
# 9. ENTRENAR CON MLFLOW
# ============================================================

with mlflow.start_run(run_name="DDPM-pixel"):

    mlflow.log_param("T", T)
    mlflow.log_param("beta_schedule", "linear")
    mlflow.log_param("batch_size", BATCH_SIZE)
    mlflow.log_param("epochs", EPOCHS)
    mlflow.log_param("lr", LR)

    global_step = 0
    for epoch in range(1, EPOCHS+1):
        epoch_losses = []
        for batch_x in train_dataset:
            loss = train_step(batch_x)
            epoch_losses.append(loss.numpy())
            global_step += 1

        mean_loss = float(np.mean(epoch_losses))
        print(f"Epoch {epoch}/{EPOCHS} - loss: {mean_loss:.5f}")
        mlflow.log_metric("train_loss", mean_loss, step=epoch)

    # ========================================================
    # 10. GENERAR MUESTRAS (GRID 8x8)
    # ========================================================
    samples = sample_ddpm(num_samples=64).numpy()

    os.makedirs("artifacts_exp2", exist_ok=True)

    fig, axes = plt.subplots(8, 8, figsize=(8,8))
    idx = 0
    for i in range(8):
        for j in range(8):
            axes[i, j].imshow(samples[idx])
            axes[i, j].axis("off")
            idx += 1
    grid_path = "artifacts_exp2/generated_grid.png"
    plt.tight_layout()
    plt.savefig(grid_path)
    plt.close(fig)
    mlflow.log_artifact(grid_path)

    # ========================================================
    # 11. HISTOGRAMAS DE INTENSIDAD (REALES vs GENERADAS)
    # ========================================================
    real_flat = images.reshape(-1)
    gen_flat = samples.reshape(-1)

    fig = plt.figure(figsize=(8,4))
    plt.hist(real_flat, bins=50, density=True, alpha=0.5, label="Real")
    plt.hist(gen_flat, bins=50, density=True, alpha=0.5, label="Generada")
    plt.legend()
    plt.title("Distribución de intensidades RGB")
    hist_path = "artifacts_exp2/hist_intensity.png"
    plt.savefig(hist_path)
    plt.close(fig)
    mlflow.log_artifact(hist_path)

    # ========================================================
    # 12. PCA REAL vs GENERADAS + FID-LIKE CASERO
    # ========================================================
    # Usamos n_feat reales y generadas para comparar
    n_feat_compare = min(2000, images.shape[0], samples.shape[0])
    real_sample = images[:n_feat_compare].reshape(n_feat_compare, -1)
    gen_sample = samples[:n_feat_compare].reshape(n_feat_compare, -1)

    pca = PCA(n_components=20)
    real_feats = pca.fit_transform(real_sample)
    gen_feats = pca.transform(gen_sample)

    # 2D para plot
    real_2d = real_feats[:, :2]
    gen_2d = gen_feats[:, :2]

    fig = plt.figure(figsize=(6,6))
    plt.scatter(real_2d[:,0], real_2d[:,1], s=5, alpha=0.3, label="Real")
    plt.scatter(gen_2d[:,0], gen_2d[:,1], s=5, alpha=0.3, label="Generada")
    plt.legend()
    plt.title("PCA 2D — Reales vs Generadas")
    pca_path = "artifacts_exp2/pca_real_vs_gen.png"
    plt.savefig(pca_path)
    plt.close(fig)
    mlflow.log_artifact(pca_path)

    # FID-like casero (no FID formal)
    mu_real = real_feats.mean(axis=0)
    mu_gen = gen_feats.mean(axis=0)
    cov_real = np.cov(real_feats, rowvar=False)
    cov_gen = np.cov(gen_feats, rowvar=False)

    mean_diff = np.linalg.norm(mu_real - mu_gen)
    cov_diff = np.linalg.norm(cov_real - cov_gen, ord="fro")
    fid_like = float(mean_diff + cov_diff)

    mlflow.log_metric("fid_like_pca20", fid_like)

    # Guardar stats como artefacto
    np.savez(
        "artifacts_exp2/stats_pca20.npz",
        mu_real=mu_real,
        mu_gen=mu_gen,
        cov_real=cov_real,
        cov_gen=cov_gen,
        fid_like=fid_like,
    )
    mlflow.log_artifact("artifacts_exp2/stats_pca20.npz")

    # ========================================================
    # 13. GUARDAR MODELO
    # ========================================================
    model_path = "artifacts_exp2/ddpm_unet_model.keras"
    model.save(model_path)
    mlflow.log_artifact(model_path)

print("=== EXP2 (DDPM) COMPLETADO ===")

Dataset: (89400, 16, 16, 3)


Model: "functional_4"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ t (InputLayer)      │ (None)            │          0 │ -                 │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ x (InputLayer)      │ (None, 16, 16, 3) │          0 │ -                 │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ time_embedding_5    │ (None, 64)        │          0 │ t[0][0]           │
│ (TimeEmbedding)     │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_46 (Conv2D)  │ (None, 16, 16,    │        896 │ x[0][0]           │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_23 (Dense)    │ (None, 64)        │      4,160 │ time_embedding_5… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_47 (Conv2D)  │ (None, 16, 16,    │      9,248 │ conv2d_46[0][0]   │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_24 (Dense)    │ (None, 64)        │      4,160 │ dense_23[0][0]    │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ add_time_layer_12   │ (None, 16, 16,    │      2,080 │ conv2d_47[0][0],  │
│ (AddTimeLayer)      │ 32)               │            │ dense_24[0][0]    │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d_8     │ (None, 8, 8, 32)  │          0 │ add_time_layer_1… │
│ (MaxPooling2D)      │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_48 (Conv2D)  │ (None, 8, 8, 64)  │     18,496 │ max_pooling2d_8[… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_49 (Conv2D)  │ (None, 8, 8, 64)  │     36,928 │ conv2d_48[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ add_time_layer_13   │ (None, 8, 8, 64)  │      4,160 │ conv2d_49[0][0],  │
│ (AddTimeLayer)      │                   │            │ dense_24[0][0]    │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d_9     │ (None, 4, 4, 64)  │          0 │ add_time_layer_1… │
│ (MaxPooling2D)      │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_50 (Conv2D)  │ (None, 4, 4, 128) │     73,856 │ max_pooling2d_9[… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_51 (Conv2D)  │ (None, 4, 4, 128) │    147,584 │ conv2d_50[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ add_time_layer_14   │ (None, 4, 4, 128) │      8,320 │ conv2d_51[0][0],  │
│ (AddTimeLayer)      │                   │            │ dense_24[0][0]    │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ up_sampling2d_8     │ (None, 8, 8, 128) │          0 │ add_time_layer_1… │
│ (UpSampling2D)      │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate_8       │ (None, 8, 8, 192) │          0 │ up_sampling2d_8[… │
│ (Concatenate)       │                   │            │ add_time_layer_1… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_52 (Conv2D)  │ (None, 8, 8, 64)  │    110,656 │ concatenate_8[0]… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_53 (Conv2D)  │ (None, 8, 8, 64)  │     36,928 │ conv2d_52[0][0] 

 Total params: 495,267 (1.89 MB)

 Trainable params: 495,267 (1.89 MB)

 Non-trainable params: 0 (0.00 B)

None
Epoch 1/50 - loss: 0.41988
Epoch 2/50 - loss: 0.22218
Epoch 3/50 - loss: 0.18726
Epoch 4/50 - loss: 0.17127
Epoch 5/50 - loss: 0.15929
Epoch 6/50 - loss: 0.15265
Epoch 7/50 - loss: 0.14653
Epoch 8/50 - loss: 0.14182
Epoch 9/50 - loss: 0.13546
Epoch 10/50 - loss: 0.13336
Epoch 11/50 - loss: 0.12965
Epoch 12/50 - loss: 0.12742
Epoch 13/50 - loss: 0.12314
Epoch 14/50 - loss: 0.12236
Epoch 15/50 - loss: 0.11938
Epoch 16/50 - loss: 0.11771
Epoch 17/50 - loss: 0.11630
Epoch 18/50 - loss: 0.11363
Epoch 19/50 - loss: 0.11198
Epoch 20/50 - loss: 0.11164
Epoch 21/50 - loss: 0.10997
Epoch 22/50 - loss: 0.10872
Epoch 23/50 - loss: 0.10800
Epoch 24/50 - loss: 0.10724
Epoch 25/50 - loss: 0.10444
Epoch 26/50 - loss: 0.10312
Epoch 27/50 - loss: 0.10265
Epoch 28/50 - loss: 0.10183
Epoch 29/50 - loss: 0.10103
Epoch 30/50 - loss: 0.09974
Epoch 31/50 - loss: 0.09973
Epoch 32/50 - loss: 0.09900
Epoch 33/50 - loss: 0.09807
Epoch 34/50 - loss: 0.09778
Epoch 35/50 - loss: 0.09647
Epoch 36/50 - loss: 0.09